<a href="https://colab.research.google.com/github/SeoYeonJu4382/bioinfo_1_project_yjseo/blob/main/Final_Free_Proj_2022_24764.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# < 생물정보학 및 실습 1 - Final Free Project >
##2022-24764 서연주
### 1. Thema : Random forest를 이용하여 LIN28A binding motif predictor 만들기

### 2. Tools : Running from Mac and using python and R

###3. Object :

CLIP-seq로 LIN28A-RNA complex를 구현 함으로써 LIN28A의 binding motif에 대한 특징을 파악할 수 있다.
LIN28A-bound sequence들로 기계학습으로 예측 모델을 만들고 평가해 봄으로써 key sequence들을 파악할 수 있다.
LIN28A binding motif predictor를 Random forest 모델을 적용하여 생성하고 학습시킬 수 있다.
Random forest 모델링을 진행해 봄으로써 모델에 대한 이해와 방법론에 대한 원리에 대한 이해를 넓힐 수 있다.
### Final Free Project용 환경 셋업

```
생물정보학 및 실습 1

이 노트북은 Creative Commons BY-SA 조건으로 재사용 가능합니다.
장혜식 <hyeshik@snu.ac.kr>, 서울대학교 생명과학부
```



## Google Drive 마운트

데이터 파일들을 불러오고 저장하기 위해서 Google Drive를 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 실습용 Bioconda 환경 설치하기


In [ ]:
!git clone https://github.com/hyeshik/colab-biolab.git
!cd colab-biolab && bash tools/setup.sh
exec(open('colab-biolab/tools/activate_conda.py').read())

Cloning into 'colab-biolab'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 69 (delta 22), reused 54 (delta 13), pack-reused 0
Unpacking objects: 100% (69/69), done.
./
./root/
./root/.bashrc.biolab
./root/.bin.priority/
./root/.bin.priority/pip
./root/.bin.priority/pip2
./root/.bin.priority/pip3
./root/.condarc
./root/.profile
./root/.tmux.conf
./root/.vimrc
--2022-06-08 06:04:55--  https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 76607678 (73M) [application/x-sh]
Saving to: ‘miniconda3.sh’

miniconda3.sh       100%[===================>]  73.06M  31.9MB/s    in 2.3s    

2022-06-08 06:04:58 (31.9 MB/s) - ‘miniconda3.sh’ sav

설정이 끝났으면, 이제 콘솔을 열어서 터미널로 접속할 수 있는 통로를 열어줍니다.

## Google Drive에 시퀀싱 데이터 다운로드: 


In [ ]:
!wget -O - --no-check-certificate https://hyeshik.qbio.io/binfo/binfo1-datapack1.tar | tar -C /content/drive/MyDrive -xf -

--2022-04-29 06:03:56--  https://hyeshik.qbio.io/binfo/binfo1-datapack1.tar
Resolving hyeshik.qbio.io (hyeshik.qbio.io)... 147.47.203.129
Connecting to hyeshik.qbio.io (hyeshik.qbio.io)|147.47.203.129|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 6424668160 (6.0G) [application/octet-stream]
Saving to: ‘STDOUT’

-                    16%[==>                 ]   1.00G  2.49MB/s    in 6m 3s   

2022-04-29 06:10:00 (2.84 MB/s) - Connection closed at byte 1078050517. Retrying.

--2022-04-29 06:10:01--  (try: 2)  https://hyeshik.qbio.io/binfo/binfo1-datapack1.tar
Connecting to hyeshik.qbio.io (hyeshik.qbio.io)|147.47.203.129|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 206 Partial Content
Length: 6424668160 (6.0G), 5346617643 (5.0G) remaining [application/octet-stream]
Saving to: ‘STDOUT’

-                    33%[+++==>              ]   2.01G  2.63MB

받은 데이터의 MD5 체크섬을 확인합니다. 한참 걸립니다. 각 파일들의 원래 체크섬은 다음과 같습니다. 한 번 비교해 보세요.

```
140aaf30bcb9276cc716f8699f04ddd6  CLIP-35L33G.bam
f1b3336ed7e2f97d562dcc71641251bd  CLIP-35L33G.bam.bai
328883a73d507eafbf5b60bd6b906201  RNA-control.bam
02073818e2f398a73c3b76e5169de1ca  RNA-control.bam.bai
b09550d09d6c2a4ce27f0226f426fdb1  RNA-siLin28a.bam
fef112c727244060ea62d3f2564a07f6  RNA-siLin28a.bam.bai
28bbd0c47d725669340c784f1b772c01  RNA-siLuc.bam
43590fdc4d81905c0432e0d1cb8cfd5b  RNA-siLuc.bam.bai
5c08a9297307bc83259e658c4474f0cc  RPF-siLin28a.bam
a1bb3e29be412dfd7fd8d16b1b1acc4c  RPF-siLin28a.bam.bai
f2eebf50943024d0116c9cd3e744c707  RPF-siLuc.bam
dc24f69e8f571fc8be30f28ce5b84fcd  RPF-siLuc.bam.bai
```

In [ ]:
!md5sum drive/MyDrive/binfo1-datapack1/*

140aaf30bcb9276cc716f8699f04ddd6  drive/MyDrive/binfo1-datapack1/CLIP-35L33G.bam
f1b3336ed7e2f97d562dcc71641251bd  drive/MyDrive/binfo1-datapack1/CLIP-35L33G.bam.bai
d54d428f5872030c4ffc43058709b896  drive/MyDrive/binfo1-datapack1/gencode.gtf
794eb8d273987a094dcc013afa2a1956  drive/MyDrive/binfo1-datapack1/gencode.gtf.gz
328883a73d507eafbf5b60bd6b906201  drive/MyDrive/binfo1-datapack1/RNA-control.bam
02073818e2f398a73c3b76e5169de1ca  drive/MyDrive/binfo1-datapack1/RNA-control.bam.bai
b09550d09d6c2a4ce27f0226f426fdb1  drive/MyDrive/binfo1-datapack1/RNA-siLin28a.bam
fef112c727244060ea62d3f2564a07f6  drive/MyDrive/binfo1-datapack1/RNA-siLin28a.bam.bai
28bbd0c47d725669340c784f1b772c01  drive/MyDrive/binfo1-datapack1/RNA-siLuc.bam
5c08a9297307bc83259e658c4474f0cc  drive/MyDrive/binfo1-datapack1/RPF-siLin28a.bam
a1bb3e29be412dfd7fd8d16b1b1acc4c  drive/MyDrive/binfo1-datapack1/RPF-siLin28a.bam.bai
f2eebf50943024d0116c9cd3e744c707  drive/MyDrive/binfo1-datapack1/RPF-siLuc.bam
dc24f69e8f571fc8b

실습에 필요한 프로그램들 몇 가지를 추가로 설치 및 디렉토리 설정

In [ ]:
!conda install -y subread

/bin/bash: conda: command not found


In [ ]:
%cd /content/drive/MyDrive/binfo1-datapack1/

/content/drive/MyDrive/binfo1-datapack1


# 1) prediction model에 넣을 input data 생성!

## 01. reference data 불러오기

In [ ]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 5.2 MB/s 


In [ ]:
from Bio import SeqIO

for seq_record in SeqIO.parse("GRCm39.primary_assembly.genome.fa", "fasta"):
  print(seq_record.id)
  print(repr(seq_record.seq))
  print(len(seq_record))

chr1
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
195154279
chr2
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
181755017
chr3
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
159745316
chr4
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
156860686
chr5
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
151758149
chr6
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
149588044
chr7
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
144995196
chr8
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
130127694
chr9
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
124359700
chr10
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
130530862
chr11
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
121973369
chr12
Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')
120092757
c

In [ ]:
for seq_record in SeqIO.parse("GRCm39.primary_assembly.genome.fa", "fasta"):
  if seq_record.id =='chr1':
    seq = seq_record.seq
    print(repr(seq_record.seq))

Seq('NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...NNN')


## 02. Pileup data 생성

In [ ]:
!samtools view -b -o CLIP-chr1.bam CLIP-35L33G.bam chr1
!samtools view CLIP-chr1.bam | wc -l

2912078


In [ ]:
!samtools view CLIP-chr1.bam | head -5

SRR458758.23028115	0	chr1	3056473	0	20M	*	0	0	GAATGGAAGTTCAAGGATCT	HHGHHGE@GGHHED?GEGDG	MD:Z:20	NH:i:40	HI:i:1	NM:i:0	SM:i:0	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:-
SRR458758.23879202	0	chr1	3056473	0	20M	*	0	0	GAATGGAAGTTCAAGGATCT	GBGGEEAAF=CEEDEGBGG>	MD:Z:20	NH:i:40	HI:i:1	NM:i:0	SM:i:0	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:-
SRR458758.23893532	0	chr1	3056473	0	20M	*	0	0	GAATGGAAGTTCAAGGATCT	G@?GGE=EFBDDGGGGD@GG	MD:Z:20	NH:i:40	HI:i:1	NM:i:0	SM:i:0	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:-
SRR458758.26501547	0	chr1	3056473	0	20M	*	0	0	GAATGGAAGTTCAAGGATCT	IIIHIIIHIGHIIIHEHIHH	MD:Z:20	NH:i:40	HI:i:1	NM:i:0	SM:i:0	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:-
SRR458758.685997	16	chr1	3059006	0	22M	*	0	0	TTCATTTACAGAATGGAATACT	EG7:776DGGGBD<GGEBGEG@	MD:Z:22	NH:i:30	HI:i:1	NM:i:0	SM:i:0	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:+


In [ ]:
!samtools sort -o CLIP-chr1.sorted.bam CLIP-chr1.bam
!samtools index CLIP-chr1.sorted.bam

In [ ]:
!samtools mpileup CLIP-chr1.bam > CLIP-chr1.pileup
!wc -l CLIP-chr1.pileup

[mpileup] 1 samples in 1 input files
77960011 CLIP-chr1.pileup


In [ ]:
!head CLIP-chr1.pileup

chr1	3056473	N	4	^!G^!G^!G^!G	HGGI
chr1	3056474	N	4	AAAA	HB@I
chr1	3056475	N	4	AAAA	GG?I
chr1	3056476	N	4	TTTT	HGGH
chr1	3056477	N	4	GGGG	HEGI
chr1	3056478	N	4	GGGG	GEEI
chr1	3056479	N	4	AAAA	EA=I
chr1	3056480	N	4	AAAA	@AEH
chr1	3056481	N	4	GGGG	GFFI
chr1	3056482	N	4	TTTT	G=BG


In [ ]:
!awk '$2 { print $0; }' CLIP-chr1.pileup > CLIP-chr1-gene.pileup
!tail CLIP-chr1-gene.pileup

chr1	195052804	N	3	AAA	DHI
chr1	195052805	N	3	CCC	IHI
chr1	195052806	N	3	GGG	IHH
chr1	195052807	N	3	GGG	IBI
chr1	195052808	N	3	GGG	HDF
chr1	195052809	N	2	AA	BD
chr1	195052810	N	3	GGG	IDH
chr1	195052811	N	3	GGG	IAF
chr1	195052812	N	3	AAA	F>F
chr1	195052813	N	3	A$A$A$	GFI


In [ ]:
import pandas as pd

pileup = pd.read_csv('CLIP-chr1-gene.pileup', sep='\t', names=['chrom', 'pos', '_ref', 'count', 'basereads', 'quals'])
pileup.tail()

,chrom,pos,_ref,count,basereads,quals
77960006,chr1,195052809,N,2,AA,BD
77960007,chr1,195052810,N,3,GGG,IDH
77960008,chr1,195052811,N,3,GGG,IAF
77960009,chr1,195052812,N,3,AAA,F>F
77960010,chr1,195052813,N,3,A$A$A$,GFI


In [ ]:
pileup_long_read = pileup[pileup['count'] >= 50]
pileup_long_read.tail()

,chrom,pos,_ref,count,basereads,quals
77959455,chr1,195005411,N,57,ccccccccccccccccccccccccccc$c$cccccccccccccccc...,IFHIHIIBIDEIIGEI=IGGBGGIIHI:?BDI=FGIIG?IIIGG?H...
77959456,chr1,195005412,N,56,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,IHHIGIIEHHEIIGEIBIHGBIGIII;BHIBFGIID?IIIED7EGH...
77959457,chr1,195005413,N,56,tttttttttttttttttttttttttttttttttttttttttttttt...,IHGIHIIBIHEIIEDI;IHGBIGIIIAEHIGDFIHG4IIIDG/HDH...
77959458,chr1,195005414,N,55,tttttttttttttttttttttttttttttttttttttttttttttt...,IHHIGHIEIH@II:8H=IHG;HDIII>BHIGDGIIEIHIAB7HBHG...
77959459,chr1,195005415,N,56,ccccccccccc$c$c$c$c$c$c$c$c$c$c$c$c$c$c$c$c$c$...,IHHIHIIEIH@HIDEBBI=GBDBHIICD@IDGGIGGAIIIGD5DGH...


pileup에서 계산에 사용될 match와 substitution만 남기고 다른 tag들은 제거

In [ ]:
import re
toremove = re.compile('[<>$*#^]')
pileup_long_read['matches'] = pileup_long_read['basereads'].apply(lambda x: toremove.sub('', x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


match와 substitution만 남긴 데이터 생성

In [ ]:
pileup_long_read[['chrom', 'pos', 'matches']]

,chrom,pos,matches
4495,chr1,3222722,CCCAAAAAAAAAAAAAAAAAAAAAAAAA!A!A!A!A!A!A!A!A!A...
4496,chr1,3222723,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...
4497,chr1,3222724,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...
4498,chr1,3222725,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...
4499,chr1,3222726,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...
...,...,...,...
77959455,chr1,195005411,cccccccccccccccccccccccccccccccccccccccccccccc...
77959456,chr1,195005412,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...
77959457,chr1,195005413,tttttttttttttttttttttttttttttttttttttttttttttt...
77959458,chr1,195005414,tttttttttttttttttttttttttttttttttttttttttttttt...


In [ ]:
pileup_long_read.to_csv('pileup_long_read.csv') # model input sequencce 생성! 

## 03. 각 position별로 base수를 세기 

In [ ]:
pileup_long_read_data = pileup_long_read[['chrom', 'pos', 'matches']]
pileup_long_read_data['base'] = pileup_long_read_data['matches'].str.len()

pileup_long_read_data.head(10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,chrom,pos,matches,base
4495,chr1,3222722,CCCAAAAAAAAAAAAAAAAAAAAAAAAA!A!A!A!A!A!A!A!A!A...,220
4496,chr1,3222723,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,167
4497,chr1,3222724,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,166
4498,chr1,3222725,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,157
4499,chr1,3222726,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,156
4500,chr1,3222727,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,156
4501,chr1,3222728,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,157
4502,chr1,3222729,TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT...,158
4503,chr1,3222730,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,158
4504,chr1,3222731,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,157


# 04. 각 position별로 Shannon entropy를 계산

In [ ]:
# Shannon entropy 계산 함수

import collections
import math
 
 
def estimate_shannon_entropy(dna_sequence):
    m = len(dna_sequence)
    bases = collections.Counter([tmp_base for tmp_base in dna_sequence])
 
    shannon_entropy_value = 0
    for base in bases:
        # number of residues
        n_i = bases[base]
        # n_i (# residues type i) / M (# residues in column)
        p_i = n_i / float(m)
        entropy_i = p_i * (math.log(p_i, 2))
        shannon_entropy_value += entropy_i
 
    return shannon_entropy_value * -1

In [ ]:
pileup_long_read_data['entropy'] = pileup_long_read_data['matches'].map(lambda x : estimate_shannon_entropy(x))
pileup_long_read_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,chrom,pos,matches,base,entropy
4495,chr1,3222722,CCCAAAAAAAAAAAAAAAAAAAAAAAAA!A!A!A!A!A!A!A!A!A...,220,1.080934
4496,chr1,3222723,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,167,0.545662
4497,chr1,3222724,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,166,0.351828
4498,chr1,3222725,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,157,0.203572
4499,chr1,3222726,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,156,1.401164
...,...,...,...,...,...
77959455,chr1,195005411,cccccccccccccccccccccccccccccccccccccccccccccc...,57,0.485461
77959456,chr1,195005412,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,56,-0.000000
77959457,chr1,195005413,tttttttttttttttttttttttttttttttttttttttttttttt...,56,-0.000000
77959458,chr1,195005414,tttttttttttttttttttttttttttttttttttttttttttttt...,55,0.131107


In [ ]:
pileup_long_read_data['end'] = pileup_long_read_data['pos'] +  pileup_long_read['basereads'].apply(len)
pileup_long_read_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,chrom,pos,matches,base,entropy,end
4495,chr1,3222722,CCCAAAAAAAAAAAAAAAAAAAAAAAAA!A!A!A!A!A!A!A!A!A...,220,1.080934,3223038
4496,chr1,3222723,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGG...,167,0.545662,3222911
4497,chr1,3222724,AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA...,166,0.351828,3222901
4498,chr1,3222725,GGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGGAGGAGGGGGG...,157,0.203572,3222882
4499,chr1,3222726,GGGAAAAGGAAAGGGGGAAAAGAGGGAGCAAGGAGGGGAGGAGGAG...,156,1.401164,3222882
...,...,...,...,...,...,...
77959455,chr1,195005411,cccccccccccccccccccccccccccccccccccccccccccccc...,57,0.485461,195005470
77959456,chr1,195005412,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...,56,-0.000000,195005468
77959457,chr1,195005413,tttttttttttttttttttttttttttttttttttttttttttttt...,56,-0.000000,195005469
77959458,chr1,195005414,tttttttttttttttttttttttttttttttttttttttttttttt...,55,0.131107,195005469


In [ ]:
# 계산한 Shannon entropy를 bed format 으로 출력해보기

chr1_bedgraph_pileup_data = pileup_long_read_data[['chrom','pos','end','entropy']]
chr1_bedgraph_pileup_data.columns = ['chromosome','start','end', 'entropy' ]

chr1_bedgraph_pileup_data

,chromosome,start,end,entropy
4495,chr1,3222722,3223038,1.080934
4496,chr1,3222723,3222911,0.545662
4497,chr1,3222724,3222901,0.351828
4498,chr1,3222725,3222882,0.203572
4499,chr1,3222726,3222882,1.401164
...,...,...,...,...
77959455,chr1,195005411,195005470,0.485461
77959456,chr1,195005412,195005468,-0.000000
77959457,chr1,195005413,195005469,-0.000000
77959458,chr1,195005414,195005469,0.131107


### select entropy >0.7 -> 선행논문 참고

In [ ]:
chr1_bedgraph_pileup_data =  pd.read_csv('chr1_bedgraph.bed', sep='\t',names=['chrom','pos','end','entropy'])
chr1_bedgraph_pileup_data.head()

,chrom,pos,end,entropy
0,chr1,3222722,3223038,1.080934
1,chr1,3222723,3222911,0.545662
2,chr1,3222724,3222901,0.351828
3,chr1,3222725,3222882,0.203572
4,chr1,3222726,3222882,1.401164


In [ ]:
chr1_bedgraph_pileup_data = chr1_bedgraph_pileup_data[chr1_bedgraph_pileup_data['entropy'] >= 0.7]
chr1_bedgraph_pileup_data.head()

,chrom,pos,end,entropy
0,chr1,3222722,3223038,1.080934
4,chr1,3222726,3222882,1.401164
28,chr1,3224558,3224845,0.978449
45,chr1,3224575,3224780,1.057954
46,chr1,3224576,3224744,1.261116


In [ ]:
chr1_bedgraph_pileup_data.to_csv("chr1_bedgraph_0_7.bed",header=None, index=False, sep='\t')

### Model의 최종 input sequence data 생성 !

In [ ]:
!bedtools getfasta -fi "GRCm39.primary_assembly.genome.fa" -bed "chr1_bedgraph_0_7.bed" -fo chr1.fa.out

In [ ]:
!head chr1.fa.out

>chr1:3222722-3223038
GAGGCGTAGACAGTTATATTGTTTCTCTAGAATCGGTCCTCATTACAGGAGGGCCAAGATACTCAGATTAAAAAGTATTTTACGTTTGAGTCAATGCAGGGCTCTGGGCAACCCCAGAGCAGCTTGTGGCATTTCTTTTGTTTTGCAAACAGTGCCTGAGAAAGTTTTTTCCCTGTGTTCAAGAGAAATTCTTTTTAACAGTGTTGCAGATCTATTCAGATGTTTAATAATGCTTAAATTCAAAGAGTTTTGCTTCTAGTGAACTGTAACCACTAGAAAATTTTGCCTCTAGGTATGGCTAATGTAACTTTACATT
>chr1:3222726-3222882
CGTAGACAGTTATATTGTTTCTCTAGAATCGGTCCTCATTACAGGAGGGCCAAGATACTCAGATTAAAAAGTATTTTACGTTTGAGTCAATGCAGGGCTCTGGGCAACCCCAGAGCAGCTTGTGGCATTTCTTTTGTTTTGCAAACAGTGCCTGAG
>chr1:3224558-3224845
AGATGAAGATGGAGCGCGGTGGCTGCCAGAGAGATTAATTCGTCAGATGAACACAGATTCTGACTCTTCTGGTAAGTATCATTCTAAAGACTAAAATTCCTTTTGTGCTTAGAATTCAGCTGAGAGCGAAAGCTCCCAAAGCTGTTTTCCAGCTACTCTCTGAGCCAGCTCCCGACAGGAGGCCGGAGACTAGCCTCAGCTTTACAATTTGCATTTGAATAAAGTACCTAGACTTCCCCGAAAGAAGTTCTGCTTTCCTACTTTCTCACTGTCGAGATTTTGTCTTT
>chr1:3224575-3224780
GGTGGCTGCCAGAGAGATTAATTCGTCAGATGAACACAGATTCTGACTCTTCTGGTAAGTATCATTCTAAAGACTAAAATTCCTTTTGTGCTTAGAATTCAGCTGAGAGCGAAAGCTCCCAAAGCTGTTTTCCAGCTACTCTCTGAGCCA

# 2) null sequence 만들기
### random 하게 각 염기별 0.25(1/4) 확률로 임의의 null sequence 생성 

In [ ]:
from random import Random

def RNAseqGenerator(number, length):
  codeFile = open('null3_seq.txt', 'w')
  if number <= 0:
        return 'wrong number'
  else:
    base = 'ATGC'      
    random = Random()
    for j in range(1, number + 1):
      str = ''
      for i in range(1, length+1):
        index = random.randint(1,len(base))
        str = str + base[index -1]
      codeFile.write(str+'\n')

In [ ]:
# 100개 길이의, null 모델에 쓰일 random seq 서열 100개 만들기 
RNAseqGenerator(41357, 100)

In [ ]:
!head null3_seq.txt

GTTTTACCAGACCAAAGCGAGTCCGCATGCCTCCCCTTAGTTGTGTAGACAAGCACGAGACCTCGGATTAAGTACGACGCCGTGTGCTGCTTCCCGATAC
AGTGCAACCTATTGGTAGGACTAGATTTCTTGGCCTAAGGGCGGGCAGAAAACCGCCCTAAGTACTTTCGGCCGATGCATGAGGCCTGAACTCGTTATCG
CGGTCTTCTTGTGCTGTAGCTTCCGCAGACATGATCGGTGACGTGAGACGGCACACAACCGGTGACCCGTCGAAGACGGGCTACGGCCTCGCCGAAATCC
AGATGTATGTCCTGATGGAATCGGATGCTACTGAACCTCTGTACCTCATTTAGTATTCTTCCGGTTATTCGCAGTATATAATTCTTAGATCAGCCCTAGG
TAACCCCTCAGAATTAAATAACTGAAACGGGGTGTTGGTTGCGAGCAATGGAATCCTCCCCCAAACACCGAGTCAATTCTTCAGAGATATCGATAGCATT
TGGTCGAGTACAGGTTCTCATCAGCAAGACCCGCTCCGTTTCGCCACGGTTGTACACACGTTACCTCGTTTGGATATTACTCTGGGCACTGAAACCGTGA
AAAAATAGATGAACGTACGATGCCTAGGGGCCAATGGATGTGGGAATGGGTTTTACGTTCATCCGCTCTGCTGGATCAATCGTGCCGAAGATCCCCAAAC
GAAGTTCATAGAACTTCTTGCCTTCCCACTCCAACTCCCTGGGTTATTATAGCTGAAAACATGCGACAGGAAACATTCATGGGCCCAGACCCCGGGCTTG
GTGCAACTCACGAAGGTGAAATGTACGTCCTGGAGCTAGGGACTTAGTCCAAGCTGATAGCCTCACGTCTATCCCCTCACTGAGGTCGGCCCCAGTGTAA
CTTGAATAACGGTTGCGAGATAGTATCGTTACACCACGTCCAGGACCGTTTCGCTACTTTTCATGGAATTTTGAGGAGACTTGTTTCCGGT

# 3) motif predictor model 만들기 
### R 이용
### random forest 모델 사용  

#4) model 성능 평가 하기 
### R 이용
### confusion matrix